In [31]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf

C:\Anaconda_Python\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
df = pd.read_csv("C:/Users/Heet Gorakhitya/Downloads/sonar.csv")
#df.head()
X = df[df.columns[:]].values
y = df[df.columns[60]]

In [20]:
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels, n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [22]:
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)
Y = one_hot_encode(y)
#Y


In [23]:
def read_dataset():
    df = pd.read_csv("C:/Users/Heet Gorakhitya/Downloads/sonar.csv")
    X = df[df.columns[:]].values
    y = df[df.columns[60]]
    encoder = LabelEncoder()
    encoder.fit(y)
    y = encoder.transform(y)
    Y = one_hot_encode(y)    
    return (X, Y)

In [29]:
X, Y = read_dataset()
X, Y = shuffle(X, Y, random_state=1)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(165, 61)
(42, 61)
(165, 2)
(42, 2)


In [ ]:
learning_rate = 0.3
epochs = 1000
cost_history = np.empty(shape=[1], dtype=float)
n_dims = X.shape[1]
classes = 2
model_path = "C:/Users/Heet Gorakhitya/Desktop"